# Artificial Neural Network

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
tf.__version__

'2.18.0'

## Data Preprocessing

### Importing the dataset

In [4]:
import os

current_dir = os.path.dirname(os.path.realpath('__file__'))
root = os.path.join(current_dir, "Books")

data = []

# Loop through author and book folders, adding .txt file contents to data with matching labels
for author in os.listdir(root):
    author_path = os.path.join(root, author)

    for book in os.listdir(author_path):
        book_path = os.path.join(author_path, book)

        for chapter in os.listdir(book_path):
            chapter_path = os.path.join(book_path, chapter)
            
            with open(chapter_path, 'r', encoding='utf-8') as file:
                text = file.read()
                
            data.append({
                'Author': author,
                'Book': book,
                'Chapter': chapter[:-4],
                'Text': text
            })

dataset = pd.DataFrame(data)

### Splitting and Cleaning

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

fragment_size = 200
overlap = 50

# First we clean our text, converting to lower case and removing unwanted characted
# Then we apply corpus, simplifying our text
# Finally we split our text into fragments of 'fragment_size', with an overlap of 'overlap' words from the previous fragment

def preprocess_text(text, ps, all_stopwords):
    # Clean text
    text = text.lower()  # Lowercase
    text = re.sub(r'\n', " ", text)  # Newlines
    text = re.sub(r'[^a-zA-Z\s]', " ", text)  # Punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Extra spaces

    # Apply corpus
    words = text.split()
    words = [ps.stem(word) for word in words if word not in all_stopwords]
    processed_text = " ".join(words)

    return processed_text

def fragment_text(text, fragment_size, overlap):
    # Split text into fragments of fragment_size length, returns array of fragments
    words = text.split()
    current_text_fragments = []
    
    step_size = fragment_size - overlap  
    
    for i in range(0, len(words), step_size):
        current_fragment = " ".join(words[i:i + fragment_size])
        current_text_fragments.append(current_fragment)

        # Handle situation where final chapter fragment is already contained in the previous fragment
        if len(words) - i < fragment_size:
            break
        
    return current_text_fragments

In [6]:
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

# Apply our cleaning and create a new dataset to replace our previous one, this time with processed text
text_fragments = []
for index, row in dataset.iterrows():
    text = row["Text"]
    text = preprocess_text(text, ps, all_stopwords)
    current_text_fragments = fragment_text(text, fragment_size, overlap)
    
    for text_fragment in current_text_fragments:
        text_fragments.append({
            "Book": row["Book"],
            "Author": row["Author"],
            "Text": text_fragment
        })

# Convert the data fragments into a Pandas DataFrame and replace the original
dataset = pd.DataFrame(text_fragments)

### Encoding

In [7]:
X = dataset["Text"].values
y = dataset["Author"].values

In [17]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Reshape y to a 2D array (needed by OneHotEncoder)
y_reshaped = y.reshape(-1, 1)

# Create and fit the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y_reshaped)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [18]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Creating class weights (in order to reduce overfitting from imbalanced data size per author)
y_class_indices = np.argmax(y_encoded, axis=-1)

class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(y_class_indices), 
    y=y_class_indices
)

# Creating class weights dictionary
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# print(f"Class weights: {class_weight_dict}")

## Encoding

### Tokenisation and Padding

In [19]:
# Remove empty strings after .strip() applied
# There shouldn't be any but tokeniser fails if not done
X = [x for x in X if x.strip() != '']

In [20]:
from transformers import BertTokenizerFast

# Max length chosen based on token distribution coming from fragment size to minimise truncation
# Edit if changing fragment_size, or remove entirely although this should be less efficient
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
X_tokenized = tokenizer(
    X, 
    padding=True, 
    truncation=True,
    max_length=330,
    return_tensors='tf'
)

C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from transformers import TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from tensorflow.keras.models import Model

# Load pretrained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def create_model():
    # Define input layers
    input_ids = tf.keras.layers.Input(shape=(330,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(330,), dtype=tf.int32, name="attention_mask")
    
    # Wrap BERT in a Lambda layer to handle KerasTensors correctly
    def bert_layer(inputs):
        return bert_model(input_ids=inputs[0], attention_mask=inputs[1])[1]
    
    bert_output = Lambda(bert_layer, output_shape=(768,))([input_ids, attention_mask])
    
    # Add dropout (prevent overfitting)
    dropout = Dropout(0.3)(bert_output)
    
    # Create classification layers
    num_authors = len(dataset['Author'].unique())
    output = Dense(num_authors, activation="softmax")(dropout)
    
    # # Create model
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
model = create_model()

# Train the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Evaluate
# val_predictions = model.predict([X_val_fold['input_ids'], X_val_fold['attention_mask']])
# val_pred_classes = val_predictions.argmax(axis=-1)
# print(f"Fold {fold + 1} Classification Report:")
# print(classification_report(np.argmax(y_val_fold, axis=-1), val_pred_classes))

In [19]:
tests_path = os.path.join(current_dir, "Tests")

test_file_path = os.path.join(tests_path, "Oscar Wilde.txt")

with open(test_file_path, 'r', encoding='utf-8') as file:
    new_text = file.read()

processed_new_text = preprocess_text(new_text, ps, all_stopwords)

X_test_tokenized = tokenizer(
    processed_new_text, 
    padding='max_length',
    truncation=True,
    max_length=330,
    return_tensors='tf'
)

In [20]:
# Make predictions on the test data
predictions = model.predict([X_test_tokenized['input_ids'], X_test_tokenized['attention_mask']])

# Get the predicted class
predicted_classes = predictions.argmax(axis=-1)  # Get the index of the highest probability

# Map to predicted class to author name
author_names = dataset['Author'].unique()
author_mapping = {index: author for index, author in enumerate(author_names)}
predicted_authors = [author_mapping[i] for i in predicted_classes]

print(predicted_authors)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
['Herman Melville']


In [16]:
# Assuming X_tokenized is your test data and y are the true labels (one-hot encoded)
loss, accuracy = model.evaluate(
    [X_tokenized['input_ids'], X_tokenized['attention_mask']],  # Model inputs
    y_encoded  # True labels
)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - accuracy: 0.0812 - loss: 2.1858    
Test Loss: 2.036719560623169
Test Accuracy: 0.11369193345308304


### Training Model

## Evaluate New Text